In [1]:
import sys
sys.path.append('..')
from polynomial_program import PolynomialProgram

In [2]:
import functools
from collections import OrderedDict

# from qiskit import IBMQ
# IBMQ.load_account()
import numpy as np
from qiskit import Aer
from qiskit.aqua import QuantumInstance, aqua_globals
from qiskit.aqua.algorithms import VQE, QAOA
from qiskit.aqua.components.optimizers import SPSA
from qiskit.circuit.library import RealAmplitudes

In [3]:
M = [2, 6, 3, 6]
K = [1, 4, 2, 8]
T = [12, 6, 24]
d = 19


def get_time_matrix(M, T):
    r = []
    for i in M:
        tmp = []
        for j in T:
            tmp.append(j / i)
        r.append(tmp)
    return np.array(r)


def get_cost_matrix(time_matrix, K):
    m = []
    for i in range(len(time_matrix)):
        tmp = []
        for j in time_matrix[i]:
            tmp.append(K[i] * j)
        m.append(tmp)
    return m


time_matrix = np.array(get_time_matrix(M, T))
cost_matrix = np.array(get_cost_matrix(time_matrix, K))

print("Time matrix:\n {}".format(time_matrix))
print("Cost matrix:\n {}".format(cost_matrix))

Time matrix:
 [[ 6.  3. 12.]
 [ 2.  1.  4.]
 [ 4.  2.  8.]
 [ 2.  1.  4.]]
Cost matrix:
 [[ 6.  3. 12.]
 [ 8.  4. 16.]
 [ 8.  4. 16.]
 [16.  8. 32.]]


In [4]:
def sample_most_likely(state_vector):
    if isinstance(state_vector, (OrderedDict, dict)):
        # get the binary string with the largest count
        binary_string = sorted(state_vector.items(), key=lambda kv: kv[1])
        repetitions = int(binary_string[-1][1])
        binary_string = binary_string[-1][0]
        x = np.asarray([int(y) for y in reversed(list(binary_string))])
        return x, repetitions
    return [], 0


optimal_key = "0000001000"


def get_stats_for_result(dict_res):
    optimal = 0
    correct = 0
    incorrect = 0
    correct_config = 0
    incorrect_config = 0

    if optimal_key in dict_res:
        optimal = dict_res[optimal_key]
    for key, val in dict_res.items():
        key = key[::-1]
        if is_correct(key):
            correct += val
            correct_config += 1
        else:
            incorrect += val
            incorrect_config += 1

    print('most likely solution: ', sample_most_likely(dict_res))
    print("optimal: ", optimal)
    print("correct solutions: ", correct)
    print("incorrect solutions: ", incorrect)
    print("correct configs: ", correct_config)
    print("incorrect configs: ", incorrect_config)


def is_correct(key):
    return solution_vector_correct(key) and execution_time(key) == d


correct_machines = ['00', '01', '10', '11']
machine_to_index = {'00': 0, '01': 1, '10': 2, '11': 3}


def solution_vector_correct(vector):
    task1_machine = vector[0:2]
    task2_machine = vector[2:4]
    task3_machine = vector[4:6]

    return task1_machine in correct_machines \
        and task2_machine in correct_machines \
        and task3_machine in correct_machines


def execution_time(k):
    task1_machine = machine_to_index.get(k[0:2])
    task2_machine = machine_to_index.get(k[2:4])
    task3_machine = machine_to_index.get(k[4:6])

    
    task1_time = time_matrix[task1_machine, 0] if task1_machine is not None else 0
    task2_time = time_matrix[task2_machine, 1] if task2_machine is not None else 0
    task3_time = time_matrix[task3_machine, 2] if task3_machine is not None else 0
    

    slack_sum = int(k[6]) * 8 + int(k[7]) * 4 + int(k[8]) * 2 + int(k[9]) * 1

    return task1_time + task2_time + task3_time + slack_sum


def execution_cost(k):
    task1_machine = machine_to_index.get(k[0:2])
    task2_machine = machine_to_index.get(k[2:4])
    task3_machine = machine_to_index.get(k[4:6])

    
    task1_cost = cost_matrix[task1_machine, 0] if task1_machine is not None else 0
    task2_cost = cost_matrix[task2_machine, 1] if task2_machine is not None else 0
    task3_cost = cost_matrix[task3_machine, 2] if task3_machine is not None else 0
    

    return task1_cost + task2_cost + task3_cost


def incorrect_machine_count(k):
    task1_machine = machine_to_index.get(k[0:2])
    task2_machine = machine_to_index.get(k[2:4])
    task3_machine = machine_to_index.get(k[4:6])


    return (0 if k[0:2] in correct_machines else 1) \
         + (0 if k[2:4] in correct_machines else 1) \
         + (0 if k[4:6] in correct_machines else 1)

In [5]:
def get_cost_model(x):
    return sum([
            cost_matrix[0, i] * (1 - x[2 * i]) * (1 - x[2 * i + 1])
            + cost_matrix[1, i] * (1 - x[2 * i]) * x[2 * i + 1]
            + cost_matrix[2, i] * x[2 * i] * (1 - x[2 * i + 1])
            + cost_matrix[3, i] * x[2 * i] * x[2 * i + 1]
            for i in range(0, 3)
    ])


def get_machine_usage_model(x):
    return 0


def get_deadline_model(x):
    time_sum = sum([
                  time_matrix[0, i] * (1 - x[2 * i]) * (1 - x[2 * i + 1])
                + time_matrix[1, i] * (1 - x[2 * i]) * x[2 * i + 1]
                + time_matrix[2, i] * x[2 * i] * (1 - x[2 * i + 1])
                + time_matrix[3, i] * x[2 * i] * x[2 * i + 1]
                for i in range(0, 3)
    ])
    slack_sum = 8 * x[6] + 4 * x[7] + 2 * x[8] + x[9]
    time_constraint = (d - time_sum - slack_sum) ** 2
    
    return time_constraint

In [6]:
def compute_eigenvalues(quibit_op):
    from qiskit.aqua.algorithms import NumPyEigensolver
    count = 1024
    eigensolver = NumPyEigensolver(qubit_op, count)
    eigensolver_result = eigensolver.compute_eigenvalues()
    print('state\t\ttime\tcost\tmachine use\tcorrect\teigenvalue')
    for eigenstate, eigenvalue in zip(eigensolver_result.eigenstates, eigensolver_result.eigenvalues):
        eigenstate, = eigenstate.sample().keys()
        eigenstate = eigenstate[::-1]
        eigenvalue = eigenvalue
        print(f'{eigenstate}\t{execution_time(eigenstate)}\t{execution_cost(eigenstate)}', end='')
        print(f'\t{incorrect_machine_count(eigenstate)}\t\t{is_correct(eigenstate)}\t{eigenvalue}')

In [7]:
A = 1
# B = 20
C = 35

In [8]:
pp = PolynomialProgram(10)

pp.add_objective(get_cost_model(pp.x), A)
# pp.add_objective(get_machine_usage_model(pp.x), B)
pp.add_objective(get_deadline_model(pp.x), C)

In [9]:
qubit_op, offset = pp.to_ising()

In [10]:
compute_eigenvalues(qubit_op)

state		time	cost	machine use	correct	eigenvalue
0001000000	19.0	22.0	0		True	(-30.93749999999917+0j)
1000000000	19.0	23.0	0		True	(-29.937499999999638+0j)
0100000010	19.0	23.0	0		True	(-29.937499999999297+0j)
1010000001	19.0	24.0	0		True	(-28.937499999999737+0j)
0110000011	19.0	24.0	0		True	(-28.937499999999396+0j)
0101000100	19.0	24.0	0		True	(-28.937499999999396+0j)
1001000010	19.0	24.0	0		True	(-28.937499999999396+0j)
0000100010	19.0	25.0	0		True	(-27.93749999999985+0j)
0000010110	19.0	25.0	0		True	(-27.937499999998714+0j)
0010100011	19.0	26.0	0		True	(-26.937500000000092+0j)
0001100100	19.0	26.0	0		True	(-26.93749999999998+0j)
0011000000	19.0	26.0	0		True	(-26.937499999999012+0j)
0001011000	19.0	26.0	0		True	(-26.937499999998956+0j)
0010010111	19.0	26.0	0		True	(-26.937499999998956+0j)
1000100100	19.0	27.0	0		True	(-25.937499999999936+0j)
0100100110	19.0	27.0	0		True	(-25.937499999999936+0j)
1000011000	19.0	27.0	0		True	(-25.937499999999197+0j)
0100011010	19.0	27.0	0		True	(-25.937

0101111101	20.0	44.0	0		False	(26.062499999999154+0j)
1001111011	20.0	44.0	0		False	(26.062499999999183+0j)
0110111100	20.0	44.0	0		False	(26.062499999999268+0j)
1001111001	18.0	44.0	0		False	(26.062499999999297+0j)
1010111010	20.0	44.0	0		False	(26.062499999999552+0j)
1010111000	18.0	44.0	0		False	(26.062499999999694+0j)
0011111001	20.0	46.0	0		False	(28.062499999998856+0j)
0011110111	18.0	46.0	0		False	(28.06249999999897+0j)
0111111011	18.0	48.0	0		False	(30.0624999999986+0j)
1011111011	20.0	48.0	0		False	(30.062499999998998+0j)
0111111101	20.0	48.0	0		False	(30.062499999999027+0j)
1011111001	18.0	48.0	0		False	(30.062499999999282+0j)
1100111001	18.0	51.0	0		False	(33.06249999999806+0j)
1100111011	20.0	51.0	0		False	(33.06249999999806+0j)
1101111011	18.0	52.0	0		False	(34.06249999999813+0j)
1110111100	20.0	52.0	0		False	(34.06249999999816+0j)
1101111101	20.0	52.0	0		False	(34.06249999999816+0j)
1110111010	18.0	52.0	0		False	(34.062499999998415+0j)
1111111101	20.0	56.0	0		False	(38.06

1001111101	22.0	44.0	0		False	(306.0624999999992+0j)
1010111100	22.0	44.0	0		False	(306.0624999999993+0j)
1001110111	16.0	44.0	0		False	(306.06249999999955+0j)
1010110110	16.0	44.0	0		False	(306.0624999999998+0j)
0011111011	22.0	46.0	0		False	(308.06249999999875+0j)
0011110101	16.0	46.0	0		False	(308.0624999999992+0j)
0111111001	16.0	48.0	0		False	(310.06249999999864+0j)
0111111111	22.0	48.0	0		False	(310.06249999999864+0j)
1011111101	22.0	48.0	0		False	(310.0624999999991+0j)
1011110111	16.0	48.0	0		False	(310.0624999999993+0j)
1100111101	22.0	51.0	0		False	(313.06249999999807+0j)
1100110111	16.0	51.0	0		False	(313.0624999999983+0j)
1101111001	16.0	52.0	0		False	(314.06249999999807+0j)
1101111111	22.0	52.0	0		False	(314.0624999999982+0j)
1110111110	22.0	52.0	0		False	(314.0624999999982+0j)
1110111000	16.0	52.0	0		False	(314.0624999999983+0j)
1111111001	16.0	56.0	0		False	(318.06249999999807+0j)
1111111111	22.0	56.0	0		False	(318.06249999999807+0j)
0000000010	23.0	21.0	0		False	(528.062

1101110111	14.0	52.0	0		False	(874.0624999999984+0j)
1110110110	14.0	52.0	0		False	(874.0624999999986+0j)
1111110111	14.0	56.0	0		False	(878.0624999999984+0j)
0000000100	25.0	21.0	0		False	(1228.062500000001+0j)
0010000101	25.0	22.0	0		False	(1229.062500000001+0j)
0001000110	25.0	22.0	0		False	(1229.062500000001+0j)
1000000110	25.0	23.0	0		False	(1230.0625000000005+0j)
0100001000	25.0	23.0	0		False	(1230.062500000001+0j)
1010000111	25.0	24.0	0		False	(1231.0625000000005+0j)
0110001001	25.0	24.0	0		False	(1231.062500000001+0j)
1001001000	25.0	24.0	0		False	(1231.062500000001+0j)
0101001010	25.0	24.0	0		False	(1231.062500000001+0j)
0000101000	25.0	25.0	0		False	(1232.0624999999995+0j)
0000010000	13.0	25.0	0		False	(1232.062500000001+0j)
0000011100	25.0	25.0	0		False	(1232.062500000001+0j)
0010101001	25.0	26.0	0		False	(1233.0624999999995+0j)
0001101010	25.0	26.0	0		False	(1233.0625+0j)
0010010001	13.0	26.0	0		False	(1233.062500000001+0j)
0001011110	25.0	26.0	0		False	(1233.062500000001+0

1110001011	27.0	32.0	0		False	(2219.0625+0j)
1101001100	27.0	32.0	0		False	(2219.0625+0j)
0111100000	11.0	32.0	0		False	(2219.0625+0j)
1011101110	27.0	32.0	0		False	(2219.0625+0j)
0111010100	11.0	32.0	0		False	(2219.062500000001+0j)
1011010010	11.0	32.0	0		False	(2219.062500000001+0j)
1100101110	27.0	35.0	0		False	(2222.062499999999+0j)
1100010010	11.0	35.0	0		False	(2222.0625+0j)
1101100000	11.0	36.0	0		False	(2223.062499999999+0j)
1101010100	11.0	36.0	0		False	(2223.0624999999995+0j)
1110101111	27.0	36.0	0		False	(2223.0624999999995+0j)
1111001100	27.0	36.0	0		False	(2223.0625+0j)
1110010011	11.0	36.0	0		False	(2223.0625+0j)
1111100000	11.0	40.0	0		False	(2227.0624999999995+0j)
1111010100	11.0	40.0	0		False	(2227.0624999999995+0j)
0000111110	27.0	41.0	0		False	(2228.062499999999+0j)
0010111111	27.0	42.0	0		False	(2229.062499999999+0j)
0001110000	11.0	42.0	0		False	(2229.0624999999995+0j)
0100110010	11.0	43.0	0		False	(2230.062499999999+0j)
1000110000	11.0	43.0	0		False	(2230.06249999

1101010000	7.0	36.0	0		False	(5023.0625+0j)
1111010000	7.0	40.0	0		False	(5027.0625+0j)
0101110000	7.0	44.0	0		False	(5031.062499999999+0j)
0111110000	7.0	48.0	0		False	(5035.062499999999+0j)
1101110000	7.0	52.0	0		False	(5039.062499999998+0j)
1111110000	7.0	56.0	0		False	(5043.062499999998+0j)
0000001011	32.0	21.0	0		False	(5883.062500000001+0j)
0001001101	32.0	22.0	0		False	(5884.062500000001+0j)
0010001100	32.0	22.0	0		False	(5884.062500000001+0j)
1000001101	32.0	23.0	0		False	(5885.0625+0j)
0100001111	32.0	23.0	0		False	(5885.0625+0j)
1010001110	32.0	24.0	0		False	(5886.0625+0j)
1001001111	32.0	24.0	0		False	(5886.0625+0j)
0000101111	32.0	25.0	0		False	(5887.0625+0j)
0011001101	32.0	26.0	0		False	(5888.062500000001+0j)
1011001111	32.0	28.0	0		False	(5890.0625+0j)
1100001111	32.0	31.0	0		False	(5893.0625+0j)
0000001100	33.0	21.0	0		False	(6828.062500000001+0j)
0010001101	33.0	22.0	0		False	(6829.062500000001+0j)
0001001110	33.0	22.0	0		False	(6829.062500000001+0j)
1000001110	33.0	23

In [11]:
seed = 10598
aqua_globals.random_seed = seed

max_trials = 1000
shots = 1000
reps = 2

entanglement = 'full'

In [12]:
spsa = SPSA(maxiter=max_trials)
ry = RealAmplitudes(qubit_op.num_qubits, reps=reps, entanglement=entanglement)
vqe = VQE(qubit_op, ry, spsa)

# provider = IBMQ.get_provider('ibm-q')
backend = Aer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(backend, seed_simulator=seed, seed_transpiler=seed, shots=shots)
result_vqe = vqe.run(quantum_instance)   
state_vector_vqe = result_vqe['eigenstate']
get_stats_for_result(state_vector_vqe)

most likely solution:  (array([1, 1, 1, 1, 1, 0, 1, 0, 0, 0]), 472)
optimal:  0
correct solutions:  938
incorrect solutions:  62
correct configs:  4
incorrect configs:  13


In [ ]:
spsa = SPSA(maxiter=max_trials)

qaoa = QAOA(qubit_op, optimizer=spsa, p=1)

# provider = IBMQ.get_provider('ibm-q')
backend = Aer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(backend, seed_simulator=seed, seed_transpiler=seed, shots=shots)
result = qaoa.run(quantum_instance)   
state_vector = result['eigenstate']
get_stats_for_result(state_vector)